In [1]:
import cmath
import numpy as np
def brizerKM(t,x,a1):
    bb=cmath.sqrt(8*a1*(1-2*a1));
    p=cmath.sqrt(2-4*a1);
    cospt=np.array([cmath.cos(p*tt) for tt in t]);
    chbx=cmath.cosh(bb*x);
    shbx=cmath.sinh(bb*x);
    N=(1-4*a1)*chbx+cmath.sqrt(2*a1)*cospt+1j*bb*shbx;
    D=(chbx-cmath.sqrt(2*a1)*cospt);
    return np.divide(N,D)*cmath.exp(1j*x);

def rbrizerKM(N,m,a):
    # [fun,Z,x]    
    h=(2*cmath.pi*m)/N;
    x=np.array([h*((x)-N/2) for x in range(N)],dtype=float);
    #x=x[:];
    Z=h;
    h2=h**2;
    h2p=h/2;
    fun= lambda t:Z*np.conj(brizerKM(x+h2p,-h2*t,a));

    return [fun,Z,x]
   
   # fun=@(t) Z*conj(brizerKM(x+h2p,-h2*t,a));

def mbrizerKM(N,m,Nt,dt,offst,a):
   # [F,X,T] [rbr,h,x]
    F=np.zeros((Nt,N),dtype=complex);
    if not offst:
        offst=0;
    if not a:
        a=1/4;
    [rbr,h,x]=rbrizerKM(N,m,a);
    h2=h**2;
    tt=np.zeros(Nt,dtype=float);
    for k in range(Nt):
        t=dt*((k-1)-Nt/2+offst);
        tt[k]=h2*t;
        F[:][k]=rbr(t);
    [T,X]=np.meshgrid(tt,x);
    return [F,X,T]



In [2]:
from NLS.nls2 import NLS,SSF,NLS_PM,NLS_PME,NLS_PMED,NLS_ex
from skimage.transform import resize
import cmath
import numpy as np
import k3d
from utils import *


def KMpbb(a1):
    bb=np.sqrt(8*a1*(1-2*a1));
    p=np.sqrt(2-4*a1);
    return [p,bb]

Period=3;
Period=4;
Period=3;
Period=5;
Period=15;
Period=12;
rep=2560/4*5;
rep=2048/4*4;

Nt=4*1000/1;
N=1*16*1024/2;
N=2*16*1024/1;
Nt=1000;

A=1-1/16
A=complex(A);
tic('start')
[xxe,tt,zz]=mbrizerKM(int(N),Period,Nt,rep,0*Nt/4,A);
toc('end')
Ta=tt[1];
n200=200;
ff=resize(np.abs(xxe),(n200,n200));
tt=resize(tt,(n200,n200));
zz=resize(zz,(n200,n200));
plot = k3d.plot(axes=['T', 'z', 'Amp'],grid_auto_fit=True,);

xmin, xmax, ymin, ymax = -5, 5, -5, 5;
cm = k3d.matplotlib_color_maps.Hsv;
z=10*ff/np.max(ff.flatten());
plt_surface = k3d.surface(z, color_map=cm, attribute=z, color_range=[-2,6],  xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
#plt_surface.grid=(-40, 40,ymin,ymax)
#plot.grid=(-40, 40,ymin,ymax,np.min(z), np.max(z));
#plot.model_matrix=[[100.,  0.,  0.,  0.],
 #        [ 0., 1.,  0.,  0.],
#         [ 0.,  0.,  1.,  0.],
#         [ 0.,  0.,  0.,  1.]];
plt_surface.model_matrix=[[10.,  0.,  0.,  0.],
         [ 0., 10.,  0.,  0.],
         [ 0.,  0.,  0.1,  0.],
         [ 0.,  0.,  0.,  1]];
plot += plt_surface;
plot.render();
plot.display();
#surf(zz,tt,abs(ff').^2);alpha 0.9
#ylabel('T');xlabel('z')
#title('brizer ex')
#;refresh;pause(0.1);
x1=xxe[1];
x2=xxe[int(np.size(xxe)/np.size(xxe[1]))-1];
dx1x2=np.linalg.norm(x1-x2)/np.linalg.norm(x1)
#stdx1=std(abs(x1))

[p,bb]=KMpbb(A)
dz=1
g=1

sname='nls-exp-dec';
sname='nls';
sname='ssf'
omagnus=0
 
mm=1/64/16/2;
#mm=1/64/16
#mm=1/16
#mm=1/8
#mm=1/32
#mm=1/2
#mm=4
#mm=1/64/2/16
#mm=1/64/4
dz=dz/mm;
nt=1;
mm_rep=mm*rep;

          
if 1:
    nls=NLS_ex(solver_name=sname,N=N,g=g,dt=dz,ftype='d');
   #   py.vncall('nls=NLS',struct('N',N,'g',g,'dt',dz,'ftype','d'));
   # py.vncall('nls=NLS_PM',struct('N',N,'g',g,'dt',dz,'ftype','d'));
   #py.vncall('nls=NLS_PME',struct('N',N,'g',g,'dt',dz,'ftype','d'));
  # py.vncall('nls=NLS_PMED',struct('N',N,'g',g,'dt',dz,'ftype','d'));
    mname=sname;
else:
    omagnus=0
    py.vncall('nls=SSF',struct('N',N,'g',g,'dt',dz,'ftype','d'));
    mname='SSF'

nls.reset(dt=dz,nt=nt,nm=[2,2],omagnus=omagnus,alpha=0*1e-8,w=1/3);

x0=xxe[0];
xxr=np.zeros((int(np.size(xxe)/np.size(xxe[1])),np.size(xxe[1])), dtype=complex);

xxr[0]=x0;

nls.x=x0;
nls.x=x0;
#tic('start')
for nT in range(1,Nt):
    xr=nls(rep=int(mm*rep),pp=1);
    xxr[nT]=xr;    
    err=np.linalg.norm(xxr[nT]-xxe[nT])/np.linalg.norm(xxe[nT]);
    #print(nT,toc,err=sprintf('%e :%3.1f%%',err,100*err));
   # print('{},{}, err={}'.format(nT,toc(''),err*100));
#toc('end')

err2=np.linalg.norm(xxr-xxe)/np.linalg.norm(xxe)
err_inf=np.linalg.norm(xxr-xxe,np.inf)/np.linalg.norm(xxe,np.inf)
err_m=np.mean(abs(xxr-xxe))/np.mean(abs(xxe))


ff=resize(np.abs(xxr),(n200,n200));
plot2 = k3d.plot();
plot2.display();
xmin, xmax, ymin, ymax = -5, 5, -5, 5;
cm = k3d.matplotlib_color_maps.Hsv;
z=10*ff/np.max(ff.flatten());
plt_surface2 = k3d.surface(z, color_map=cm, attribute=z, color_range=[-2,6],  xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
plt_surface2.grid=(xmin, xmax,ymin,ymax)

plot2 += plt_surface2
plot2.render()
#surf(tt,zz,abs(ff).^2);alpha 0.9
#ylabel('T');xlabel('z')
#surf(zz,tt,abs(ff').^2);alpha 0.9
#ylabel('T');xlabel('z')
#title(sprintf('brizer %s [magnus=%d] dz=%g',mname,omagnus,dz))

#figure;

ffe=resize(np.abs(xxr-xxe),(n200,n200));
afe=resize(np.abs(xxe),(n200,n200));
#surf(zz,tt,20*log10(ffe./afe)');alpha 0.9
#ylabel('T');xlabel('z');zlabel('Err [dB]')
mx=20*np.log10(np.divide(ffe,afe));
plot3 = k3d.plot();
plot3.display();
xmin, xmax, ymin, ymax = -5, 5, -5, 5;
cm = k3d.matplotlib_color_maps.Hsv;
z=10*mx/np.max(mx.flatten());
plt_surface3 = k3d.surface(z, color_map=cm, attribute=z, color_range=[-2,6],  xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
plt_surface3.grid=(xmin, xmax,ymin,ymax)

plot3 += plt_surface3
plot3.render()
#mx=np.max(mx)
#view([0,-1,0])
#title(sprintf('brizer %s error  [max=%3.1f dBb]   [magnus=%d] dz=%g',mname,mx,omagnus,dz))
       


pid= 15064
context_wrapper=D:\dich\2019_02_20_04-49-14\ipc\py.modules\NLS\bin/ictx/ictxwrr.dll
nls_pade_lib=D:\dich\2019_02_20_04-49-14\ipc\py.modules\NLS\bin/nls_pade.dll
ssf_lib=D:\dich\2019_02_20_04-49-14\ipc\py.modules\NLS\bin/ssf.dll
startend 27.8037 sec


D:\Programs\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

Output()

Output()

In [12]:
from utils import *
from skimage.transform import resize
import k3d
import matplotlib.pyplot as plt
from ipywidgets import interact
from ipywidgets import interact_manual
import ipyvolume as ipv

%matplotlib inline
#@interact_manual(a=(0.1, 1, 0.05))
def plot_a(a):
    Period=3;
    Period=4;
    Period=3;
    Period=5;
    Period=7;
    rep=2560/4*5;
    rep=2048/4*4;
    Nt=4*1000/1;
    N=1*16*1024/2;
    N=1*16*1024/1;
    Nt=1000;


    A=1/2.8;
    A=1/4;
    A=1/8;
    A=1-1/8
    A=1-3/4
    A=a;
    tic('start')
    [F,T,X]=mbrizerKM(int(N),Period,Nt,rep,0*Nt/4,A)
    toc(' end')
    #Ta=tt(1,:);
    xxe=np.zeros((Nt,int(N)),dtype=float);
    xxe=abs(F);
    ff=resize(xxe,(200,200));
    tt=resize(T,(200,200));
    zz=resize(X,(200,200));
    plot = k3d.plot(axes=['T', 'z', 'Amp'],grid_auto_fit=False);
   
    xmin, xmax, ymin, ymax = -40, 40, -5, 5;
    cm = k3d.matplotlib_color_maps.Hsv;
    z=10*ff/np.max(ff.flatten());
    plot.grid=(xmin,ymin,np.min(z), xmax,ymax, np.max(z));
   # plot.model_matrix=[[1.,  0.,  0.,  0.],
     #    [ 0., 10.,  0.,  0.],
     #    [ 0.,  0.,  1.,  0.],
     #    [ 0.,  0.,  0.,  1.]];
    plt_surface = k3d.surface(z, color_map=cm, attribute=z, color_range=[-2,6], xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
   
   # plt_surface.model_matrix=[[20.,  0.,  0.,  0.],
    #     [ 0., 20.,  0.,  0.],
    #     [ 0.,  0.,  1.,  0.],
    #     [ 0.,  0.,  0.,  1]];
    #plt_surface.grid=(xmin, xmax,ymin,ymax)
    plot += plt_surface
    plot.render()
    plot.display();
    print(plot.__dict__)
    ipv.figure()
    ipv.plot_surface(zz, z, tt, color="orange")
    ipv.show()
plot_a(0.75);

start end 6.58601 sec


Output()

{'_trait_values': {'_backend_version': '2.9.3', '_model_module': 'k3d', '_model_module_version': '2.9.3', '_model_name': 'PlotModel', '_view_count': None, '_view_module': 'k3d', '_view_module_version': '2.9.3', '_view_name': 'PlotView', 'antialias': 3, 'auto_rendering': True, 'axes_helper': 1.0, 'background_color': 16777215, 'camera_auto_fit': True, 'camera_fov': 60.0, 'camera_mode': 'trackball', 'camera_no_pan': False, 'camera_no_rotate': False, 'camera_no_zoom': False, 'camera_pan_speed': 0.3, 'camera_rotate_speed': 1.0, 'camera_zoom_speed': 1.2, 'colorbar_object_id': -1, 'colorbar_scientific': False, 'fps': 25.0, 'fps_meter': False, 'grid_auto_fit': False, 'grid_visible': True, 'height': 512, 'lighting': 1.5, 'manipulate_mode': 'translate', 'menu_visibility': True, 'mode': 'view', 'name': None, 'rendering_steps': 1, 'screenshot': '', 'screenshot_scale': 2.0, 'snapshot': '', 'snapshot_include_js': True, 'time': 0.0, 'voxel_paint_color': 0, 'comm': <ipykernel.comm.comm.Comm object at 